In [1]:
import pandas as pd
train = pd.read_csv('/home/ugrads/a/aa_ron_su/data/till_end_mimic_iv_extra_features_train_NOTE.csv')
test = pd.read_csv('/home/ugrads/a/aa_ron_su/data/till_end_mimic_iv_extra_features_test_NOTE.csv')

In [2]:
outpath_train = '/home/ugrads/a/aa_ron_su/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1.csv'
outpath_test = '/home/ugrads/a/aa_ron_su/data/till_end_mimic_iv_extra_features_test_NOTE_TARGET1.csv'
outpath_ft_train = '/home/ugrads/a/aa_ron_su/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT.csv'
outpath_ft_test = '/home/ugrads/a/aa_ron_su/data/till_end_mimic_iv_extra_features_test_NOTE_TARGET1_FT.csv'

In [3]:
for df in [train, test]:
    df['t_start_DT'] = pd.to_datetime(df['t_start_DT']) 

In [4]:
from datetime import timedelta
def _generate_target(subject_df):
    delta_times = subject_df[subject_df['delta'] == 1]['t_start_DT']
    # subject_df['num_deltas_per_subj'] = len(delta_times)

    if (delta_times.empty):
        return subject_df

    masks = []
    for delta_time in delta_times:
        mask1 = (subject_df['t_start_DT'] <= delta_time) 
        mask2 = (subject_df['t_start_DT'] >= (delta_time - timedelta(days = 2)))
        mask = mask1 & mask2
        masks.append(mask)
    
    mask_acc = masks[0]
    for m in masks[1:]:
        mask_acc = mask_acc | m
    
    subject_df.loc[mask_acc, 'delta_in_2_days'] = 1

    return subject_df

In [5]:
from tqdm.auto import tqdm
tqdm.pandas()


train['delta_in_2_days'] = 0
train = train.groupby('SUBJECT_ID').progress_apply(_generate_target)

test['delta_in_2_days'] = 0
test = test.groupby('SUBJECT_ID').progress_apply(_generate_target)

/home/ugrads/a/aa_ron_su/miniconda3/envs/clinical_t5/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████▉| 24716/24764 [00:43<00:00, 434.20it/s]/home/ugrads/a/aa_ron_su/miniconda3/envs/clinical_t5/lib/python3.8/site-packages/tqdm/std.py:814: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return getattr(df, df_function)(wrapper, **kwargs)
 98%|█████████▊| 4262/4344 [00:07<00:00, 571.71it/s]/home/ugrads/a/aa_ron_su/miniconda3/envs/clinical_t5/lib/pyth

In [6]:
train_note_target = train[['ICUSTAY_ID','text','delta_in_2_days','NOTE_ID']].copy()
train_note_target.dropna(inplace=True)
train_note_target = train_note_target.drop_duplicates(subset='text')

test_note_target = test[['ICUSTAY_ID','text','delta_in_2_days','NOTE_ID']].copy()
test_note_target.dropna(inplace=True)
test_note_target = test_note_target.drop_duplicates(subset='text')

In [8]:
# train.to_csv(outpath_train, index=False)
# test.to_csv(outpath_test, index=False)

train_note_target.to_csv(outpath_ft_train, index=False)
test_note_target.to_csv(outpath_ft_test, index=False)

In [66]:
test_note_target

,text,delta_in_2_days
190,\nName: ___ Unit No: ___\...,1
1104,\nName: ___ Unit No: ___...,0
1394,\nName: ___ Unit No: ___\n...,1
1553,\nName: ___ Unit No: ___\n...,1
2416,\nName: ___ Unit No: __...,0
...,...,...
1017356,\nName: ___ Unit No: __...,0
1017410,\nName: ___ Unit No: ___\n...,0
1017607,\nName: ___ Unit No: ___\...,1
1018392,\nName: ___ Unit No: ___...,0


In [52]:
def check_monotonic_increasing(icustay_df):
    if not icustay_df.t_start_DT.is_monotonic_increasing:
        print(f'NOTE_ID {icustay_df.NOTE_ID.iloc[0]} not increasing')

In [6]:
import pandas as pd
ft_train = pd.read_csv(outpath_ft_train)

In [10]:
ft_train

,ICUSTAY_ID,text,delta_in_2_days
0,30000484.0,\nName: ___ Unit No: ___\n \...,0
1,30001947.0,\nName: ___ Unit No: ___\n...,1
2,30002521.0,\nName: ___ Unit No: ___...,0
3,30003202.0,\nName: ___. Unit No: ___\n \...,0
4,30003226.0,\nName: ___ Unit No: __...,0
...,...,...,...
14568,39996870.0,\nName: ___ Unit No: ___\...,0
14569,39998012.0,\nName: ___ Unit No: ___\n \nA...,0
14570,39999230.0,\nName: ___ Unit No: __...,0
14571,39999301.0,\nName: ___ Unit No: ___\n...,1


In [ ]:
# from tqdm.auto import tqdm
# tqdm.pandas()

# df_NOTE['delta_in_2_days'] = 0
# df_NOTE = df_NOTE.groupby('SUBJECT_ID').progress_apply(_generate_target)

In [77]:
# (len(df_NOTE), len(df_NOTE_NEW))

(28724, 1293497)

In [ ]:
train

In [30]:
train_delta = train[train.delta_in_2_days != 0]
test_delta = test[test.delta_in_2_days != 0]


print(len(train),len(test))
print(len(train_delta), len(test_delta)) 
print(len(train_delta)/len(train), len(test_delta)/len(test))
print(len(train_delta.NOTE_ID.unique()), len(train.NOTE_ID.unique()))
print(len(test_delta.NOTE_ID.unique()), len(test.NOTE_ID.unique()))

3710672 1018912
226973 47609
0.061167626780270525 0.04672533054866367
3156 14574
555 2583


In [31]:
555/2583, 3156/14574

(0.2148664343786295, 0.2165500205846027)

In [87]:
(len(df_NOTE.NOTE_ID.unique()), len(df_NOTE_NEW.NOTE_ID.unique()))

(202, 10417)

In [86]:
(len(df_NOTE_delta.NOTE_ID.unique()), len(df_NOTE_NEW_delta.NOTE_ID.unique())) # ~5% of the notes have a positive target.

(18, 492)

In [16]:
df_NOTE.drop(['t_start_DT', 'SUBJECT_ID'], axis = 1, inplace=True)

In [20]:
df_NOTE.to_csv(outpath, index=False)

In [ ]:
df_NOTE

### UNIT TESTING

In [38]:
# df_NOTE = pd.read_csv(outpath)
# df_NOTE_NEW = pd.read_csv(outpath_NEW)

In [64]:
(len(df_NOTE), len(df_NOTE_NEW))

(28724, 1293497)

In [65]:
(len(df_NOTE.ICUSTAY_ID.unique()), len(df_NOTE_NEW.ICUSTAY_ID.unique()))

(446, 20522)

In [45]:
(len(df_NOTE[df_NOTE.delta != 0]), # 44
len(df_NOTE_NEW[df_NOTE_NEW.delta != 0])) # 1452

(44, 1452)

In [ ]:
pivot_table = pd.pivot_table(df_NOTE, values='ICUSTAY_ID', index='SUBJECT_ID', aggfunc=pd.Series.nunique)
pivot_table = pivot_table.rename(columns={'ICUSTAY_ID': 'NUM_ICUSTAYS'})
pivot_table[pivot_table['NUM_ICUSTAYS'] > 3]

In [ ]:
pivot_table = pd.pivot_table(df_NOTE_NEW, values='ICUSTAY_ID', index='SUBJECT_ID', aggfunc=pd.Series.nunique)
pivot_table = pivot_table.rename(columns={'ICUSTAY_ID': 'NUM_ICUSTAYS'})
pivot_table[pivot_table['NUM_ICUSTAYS'] > 3]

In [ ]:
pd.set_option('display.max_rows', 900)
x = [10035168, 18775105, 14930700]
tmp = df_NOTE_NEW[df_NOTE_NEW['SUBJECT_ID'] == x[2]] # 10035168  18775105

tmp[tmp['delta'] == 1].sort_values('t_start_DT', ascending = True)

In [ ]:
tmp.sort_values('t_start_DT', ascending = True)

In [30]:
# from datetime import timedelta
# tmp['delta_in_2_days'] = 0
# delta_time = tmp[tmp['delta'] == 1]['t_start_DT'].iloc[0]
# mask = (tmp['t_start_DT'] <= delta_time) & (tmp['t_start_DT'] >= (delta_time - timedelta(days = 2)))
# tmp.loc[mask, 'delta_in_2_days'] = 1
# # tmp.sort_values('t_start_DT', ascending = False)

/tmp/ipykernel_4119705/21244126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp['delta_in_2_days'] = 0


In [230]:
    # subject_df = tmp
    # subject_df['delta_in_2_days'] = 0
    # delta_times = subject_df[subject_df['delta'] == 1]['t_start_DT']


    # # for delta_time in delta_times:
    # masks = []
    # for delta_time in delta_times:
    #     mask1 = (subject_df['t_start_DT'] <= delta_time) 
    #     mask2 = (subject_df['t_start_DT'] >= (delta_time - timedelta(days = 2)))
    #     mask = mask1 & mask2
    #     masks.append(mask)
    
    
    # mask_acc = masks[0]
    # for m in masks[1:]:
    #     mask_acc = mask_acc | m
    
    # subject_df.loc[mask_acc, 'delta_in_2_days'] = 1

/tmp/ipykernel_3711208/1209130406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subject_df['delta_in_2_days'] = 0


In [212]:
# len(subject_df)

859

In [225]:
# [len(mask1), len(mask2), len(mask), len(mask_acc)]

[859, 859, 859, 859]

In [ ]:
# mask_acc

In [ ]:
# subject_df.sort_values('t_start_DT')

In [155]:
# mask1

pandas.core.series.Series

In [ ]:
# df_NOTE[df_NOTE['delta'] == 1]

In [ ]:
# df_NOTE[df_NOTE['delta_in_2_days'] == 1]

In [ ]:
# df_NOTE[df_NOTE.index > 180]